# Python - Structured Query Language (SQL)

A database consists of many tables that are linked.   
Each (relational database) table generally represents one entity type. such as "order" or "album".   
Each row (or record) represents an instance of the entity type, e.g. one order, one album etc.   
Each column represents an attribute of each instance, e.g. order_date, artist etc.   
Each row requires an unique identifier, known as the **primary key** so we can use to explicitly access the row in question.    
A table can also have multiple **foreign keys** for each row, used to point to specific rows (primary key of the other table). 


## 0. Workflow of SQL querying

-> Import packages and functions (e.g. sqlalchemy)   
-> Create the database engine (create_engine)   
-> Connect to the engine (con=engine.connect)   
-> Query the database (rs=con.execute(), creates a SQLAlchemy results object which we assign to rs)     
-> Save query results to a DataFrame (df=pd.DataFrame(rs.fetchall()), fetchall() fetches all rows)    
-> Close the connection (con.close())    

print(df.head()) to check


## 1. SQLAlchemy

There are a few ways (python) to connect to database, e.g. sqlite3. SQLAlchemy is used due to its compability with other Relational Database Management System, such as SQLite, MySQL and PostgreSQL.   

From the function *create_engine* from sqlalchemy, we fire up an SQL engine that will communicate our queries to the database.   
The only required argument of create_engine is a string that indicates the type of the database we are connecting to and name of the database.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine("sqlite:///Chinook.sqlite") #if DB doesnt exist, create a new DB with the said name

In [9]:
table_names = engine.table_names() #name of the tables in the DB

In [10]:
print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [22]:
con = engine.connect() #connect to the engine
rs = con.execute("SELECT * FROM Album") #execute SQL command through the connection, get the results. *=all cols
df = pd.DataFrame(rs.fetchall()) #convert result into a DF
df.columns = rs.keys()
con.close() 

In [23]:
print(df.head())

   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3


In [24]:
print(rs.keys())

['AlbumId', 'Title', 'ArtistId']


## 1.1 Using the context manager

no need to close connection

In [36]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine("sqlite:///Chinook.sqlite")
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Customer WHERE Country = 'Canada'") #or select col names instead of *,'' instead of ""
    df = pd.DataFrame(rs.fetchmany(size=5)) #fetch 5 rows
    df.columns = rs.keys()

In [37]:
print(df.head())

   CustomerId FirstName  LastName        Company                 Address  \
0           3  François  Tremblay           None       1498 rue Bélanger   
1          14      Mark   Philips          Telus          8210 111 ST NW   
2          15  Jennifer  Peterson  Rogers Canada     700 W Pender Street   
3          29    Robert     Brown           None  796 Dundas Street West   
4          30    Edward   Francis           None        230 Elgin Street   

        City State Country PostalCode              Phone                Fax  \
0   Montréal    QC  Canada    H2G 1A7  +1 (514) 721-4711               None   
1   Edmonton    AB  Canada    T6G 2C7  +1 (780) 434-4554  +1 (780) 434-5565   
2  Vancouver    BC  Canada    V6C 1G8  +1 (604) 688-2255  +1 (604) 688-8756   
3    Toronto    ON  Canada    M6J 1V1  +1 (416) 363-8888               None   
4     Ottawa    ON  Canada    K2P 1L7  +1 (613) 234-3322               None   

                 Email  SupportRepId  
0  ftremblay@gmail.com       

In [42]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Customer ORDER BY SupportRepId") #ordered by a specific col name
    df = pd.DataFrame(rs.fetchall()) 
    df.columns = rs.keys()

print(df.head())

   CustomerId FirstName   LastName  \
0           1      Luís  Gonçalves   
1           3  François   Tremblay   
2          12   Roberto    Almeida   
3          15  Jennifer   Peterson   
4          18  Michelle     Brooks   

                                            Company  \
0  Embraer - Empresa Brasileira de Aeronáutica S.A.   
1                                              None   
2                                            Riotur   
3                                     Rogers Canada   
4                                              None   

                           Address                 City State Country  \
0  Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP  Brazil   
1                1498 rue Bélanger             Montréal    QC  Canada   
2                 Praça Pio X, 119       Rio de Janeiro    RJ  Brazil   
3              700 W Pender Street            Vancouver    BC  Canada   
4                     627 Broadway             New York    NY     USA   

 

## 1.2 Use Pandas directly query database

In [45]:
engine = create_engine("sqlite:///Chinook.sqlite") #still need
df = pd.read_sql_query("SELECT * FROM Customer", engine)
print(df.head())

   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                                            Company  \
0  Embraer - Empresa Brasileira de Aeronáutica S.A.   
1                                              None   
2                                              None   
3                                              None   
4                                  JetBrains s.r.o.   

                           Address                 City State         Country  \
0  Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP          Brazil   
1          Theodor-Heuss-Straße 34            Stuttgart  None         Germany   
2                1498 rue Bélanger             Montréal    QC          Canada   
3                 Ullevålsveien 14                 Oslo  None          Norway   
4                

## 2. INNER JOIN

SELECT X , Y FROM A INNER JOIN B ON A.I = B.I    
X and Y are column in table A and B, and I is the "common" column in the the two table that links the data of the two table. *I* can be primary key in table A but foreign key in table B.   
A.I means I cols in table A

In [46]:
engine = create_engine('sqlite:///Chinook.sqlite')
with engine.connect() as con:
    rs = con.execute("SELECT Title, Name FROM Album INNER JOIN ARTIST ON Album.ArtistID = Artist.ArtistID")
    df = pd.DataFrame(rs.fetchall())
    df.columns=rs.keys()

# Print head of DataFrame df
print(df.head())

                                   Title       Name
0  For Those About To Rock We Salute You      AC/DC
1                      Balls to the Wall     Accept
2                      Restless and Wild     Accept
3                      Let There Be Rock      AC/DC
4                               Big Ones  Aerosmith


In [47]:
engine = create_engine('sqlite:///Chinook.sqlite')
df=pd.read_sql_query("SELECT * FROM PlaylistTrack INNER JOIN Track on PlaylistTrack.TrackId = Track.TrackId WHERE Milliseconds < 250000",engine)

# Print head of DataFrame
print(df.head())

   PlaylistId  TrackId  TrackId              Name  AlbumId  MediaTypeId  \
0           1     3390     3390  One and the Same      271            2   
1           1     3392     3392     Until We Fall      271            2   
2           1     3393     3393     Original Fire      271            2   
3           1     3394     3394       Broken City      271            2   
4           1     3395     3395          Somedays      271            2   

   GenreId Composer  Milliseconds    Bytes  UnitPrice  
0       23     None        217732  3559040       0.99  
1       23     None        230758  3766605       0.99  
2       23     None        218916  3577821       0.99  
3       23     None        228366  3728955       0.99  
4       23     None        213831  3497176       0.99  
